In [1]:
import os
print(os.listdir())


['clip.code-workspace', 'Untitled-1.ipynb', 'script_identifier.pth']


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision import models

In [12]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [13]:
from torchvision import datasets


In [14]:
import torchvision.transforms as transforms

In [15]:
train_dataset = datasets.ImageFolder(root="/home/pup/Desktop/Script Recognition Data/dataset/test", transform=transform)
val_dataset = datasets.ImageFolder(root="/home/pup/Desktop/Script Recognition Data/dataset/test", transform=transform)

In [16]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [17]:
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(512, len(train_dataset.classes))  # Modify for number of classes


/home/pup/anaconda3/envs/CLIP/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/pup/anaconda3/envs/CLIP/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [18]:
# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [19]:
# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [44]:
# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

In [45]:
print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")


Epoch [100/100], Loss: 0.0002


In [46]:
# Save trained model
torch.save(model.state_dict(), "script_identifier.pth")

print("Training Complete!")

Training Complete!


In [47]:
# Load pretrained ResNet-18 model
model = models.resnet18(pretrained=True)

# Modify the final classification layer to match your number of script classes
num_classes = len(train_dataset.classes)  # Automatically detects number of classes
model.fc = nn.Linear(512, num_classes)  

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(f"Model ready! Number of classes: {num_classes}")


Model ready! Number of classes: 10


In [48]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [49]:
# Training loop
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {train_acc:.2f}%")

print("Training Complete!")


Epoch [1/100], Loss: 0.6353, Accuracy: 79.77%
Epoch [2/100], Loss: 0.2732, Accuracy: 91.93%
Epoch [3/100], Loss: 0.1168, Accuracy: 96.16%
Epoch [4/100], Loss: 0.2119, Accuracy: 93.34%
Epoch [5/100], Loss: 0.1507, Accuracy: 95.90%
Epoch [6/100], Loss: 0.1080, Accuracy: 96.80%
Epoch [7/100], Loss: 0.1594, Accuracy: 94.88%
Epoch [8/100], Loss: 0.2024, Accuracy: 95.26%
Epoch [9/100], Loss: 0.1060, Accuracy: 96.67%
Epoch [10/100], Loss: 0.0830, Accuracy: 97.44%
Epoch [11/100], Loss: 0.0998, Accuracy: 97.31%
Epoch [12/100], Loss: 0.0671, Accuracy: 97.44%
Epoch [13/100], Loss: 0.0830, Accuracy: 97.70%
Epoch [14/100], Loss: 0.0908, Accuracy: 97.06%
Epoch [15/100], Loss: 0.0612, Accuracy: 98.34%
Epoch [16/100], Loss: 0.0484, Accuracy: 98.59%
Epoch [17/100], Loss: 0.0356, Accuracy: 99.10%
Epoch [18/100], Loss: 0.0433, Accuracy: 98.85%
Epoch [19/100], Loss: 0.0354, Accuracy: 98.85%
Epoch [20/100], Loss: 0.0245, Accuracy: 99.10%
Epoch [21/100], Loss: 0.0302, Accuracy: 98.98%
Epoch [22/100], Loss: 

In [50]:
torch.save(model.state_dict(), "script_identifier.pth")
print("Model saved successfully!")


Model saved successfully!


In [51]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

val_acc = 100 * correct / total
print(f"Validation Accuracy: {val_acc:.2f}%")


Validation Accuracy: 97.70%


In [81]:
from PIL import Image

# Load class names
class_names = train_dataset.classes

# Load an image for testing
def predict_script(image_path):
    image = Image.open(image_path)
    transform_test = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    
    image = transform_test(image).unsqueeze(0).to(device)  # Add batch dimension
    model.eval()
    
    with torch.no_grad():
        output = model(image)
        _, predicted = output.max(1)
    
    print(f"Predicted Script: {class_names[predicted.item()]}")

# Example usage
predict_script("/home/pup/Downloads/Punjabi/HW_WORDS_Punjabi/test/images/1.jpg")


Predicted Script: 4


In [88]:
import os
import torch
import torchvision.transforms as transforms
from PIL import Image
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load class names
class_names = train_dataset.classes

def predict_script(image_path, model, device):
    image = Image.open(image_path).convert('RGB')  # Ensure 3 channels
    transform_test = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    
    image = transform_test(image).unsqueeze(0).to(device)  # Add batch dimension
    model.eval()
    
    with torch.no_grad():
        output = model(image)
        _, predicted = output.max(1)
    
    return predicted.item()

def evaluate_model(test_folder, model, device):
    y_true = []
    y_pred = []
    
    for label in os.listdir(test_folder):
        label_path = os.path.join(test_folder, label)
        if not os.path.isdir(label_path):
            continue
        
        for img_name in os.listdir(label_path):
            img_path = os.path.join(label_path, img_name)
            if img_path.lower().endswith(('png', 'jpg', 'jpeg')):
                predicted_label = predict_script(img_path, model, device)
                y_pred.append(predicted_label)
                y_true.append(class_names.index(label))
    
    # Compute metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

# Example usage
test_folder = "/home/pup/Desktop/Script Recognition Data/dataset/test"  # Folder structure: test_folder/class_name/image.jpg
evaluate_model(test_folder, model, device)

Accuracy: 0.9795
Precision: 0.9820
Recall: 0.9795
F1 Score: 0.9796
